In [1]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 6.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
# from sodapy import Socrata # most likely won't use it
import boto3
import requests
import csv
import json
import os
import configparser
from google.cloud import bigquery

In [3]:
config = configparser.ConfigParser()
config.read('config.cfg')
#config.sections()

aws_access_key = config['default']['aws_access_key_id']
aws_secret_key = config['default']['aws_secret_access_key']

# set environment variables
os.environ['AWS_ACCESS_KEY_ID'] = aws_access_key
os.environ['AWS_SECRET_ACCESS_KEY'] = aws_secret_key

In [4]:
# Create a boto3 S3 client
s3 = boto3.client('s3')

In [5]:
s3.list_buckets()['Buckets']

[{'Name': 'aws-logs-381492128412-us-east-2',
  'CreationDate': datetime.datetime(2024, 4, 1, 13, 14, 44, tzinfo=tzlocal())},
 {'Name': 'cis-athena-queries',
  'CreationDate': datetime.datetime(2024, 5, 18, 1, 49, tzinfo=tzlocal())},
 {'Name': 'cis9760-s3-assignment-noor-mahmud',
  'CreationDate': datetime.datetime(2024, 2, 14, 2, 35, 21, tzinfo=tzlocal())},
 {'Name': 'datawarehouse2024',
  'CreationDate': datetime.datetime(2024, 4, 28, 3, 58, 21, tzinfo=tzlocal())},
 {'Name': 'noor-mahmud-datastream-bucket',
  'CreationDate': datetime.datetime(2024, 4, 13, 1, 47, 13, tzinfo=tzlocal())}]

In [6]:
s3_objects = s3.list_objects(Bucket='datawarehouse2024')

for obj in s3_objects['Contents'][:5]:
  print(obj['Key'])
  # file_obj = s3.get_object(Bucket='bucket', Key=obj['Key'])
  # df = pd.read_json(file_obj['Body'])

dw_hw/nyc_crash_data/opendata_1.json
dw_hw/nyc_crash_data/opendata_10.json
dw_hw/nyc_crash_data/opendata_11.json
dw_hw/nyc_crash_data/opendata_12.json
dw_hw/nyc_crash_data/opendata_13.json


In [47]:
for obj in s3_objects['Contents']:
  print(obj['Key'])

dw_hw/nyc_crash_data/opendata_1.json
dw_hw/nyc_crash_data/opendata_10.json
dw_hw/nyc_crash_data/opendata_11.json
dw_hw/nyc_crash_data/opendata_12.json
dw_hw/nyc_crash_data/opendata_13.json
dw_hw/nyc_crash_data/opendata_14.json
dw_hw/nyc_crash_data/opendata_15.json
dw_hw/nyc_crash_data/opendata_16.json
dw_hw/nyc_crash_data/opendata_17.json
dw_hw/nyc_crash_data/opendata_18.json
dw_hw/nyc_crash_data/opendata_19.json
dw_hw/nyc_crash_data/opendata_2.json
dw_hw/nyc_crash_data/opendata_20.json
dw_hw/nyc_crash_data/opendata_21.json
dw_hw/nyc_crash_data/opendata_22.json
dw_hw/nyc_crash_data/opendata_23.json
dw_hw/nyc_crash_data/opendata_24.json
dw_hw/nyc_crash_data/opendata_25.json
dw_hw/nyc_crash_data/opendata_26.json
dw_hw/nyc_crash_data/opendata_27.json
dw_hw/nyc_crash_data/opendata_28.json
dw_hw/nyc_crash_data/opendata_29.json
dw_hw/nyc_crash_data/opendata_3.json
dw_hw/nyc_crash_data/opendata_30.json
dw_hw/nyc_crash_data/opendata_31.json
dw_hw/nyc_crash_data/opendata_32.json
dw_hw/nyc_crash

In [7]:
response = s3.get_object(
    Bucket='datawarehouse2024',
    Key='dw_hw/nyc_crash_data/opendata_1.json',
)

#print(response)

In [8]:
df_s3 = pd.read_json(response['Body'])
print(df_s3.columns)

Index(['crash_date', 'crash_time', 'on_street_name', 'off_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'collision_id', 'vehicle_type_code1', 'vehicle_type_code2', 'borough',
       'zip_code', 'latitude', 'longitude', 'location', 'cross_street_name',
       'contributing_factor_vehicle_3', 'vehicle_type_code_3',
       'contributing_factor_vehicle_4', 'vehicle_type_code_4',
       'contributing_factor_vehicle_5', 'vehicle_type_code_5'],
      dtype='object')


In [10]:
df_s3['crash_date']

0        2021-09-11T00:00:00.000
1        2022-03-26T00:00:00.000
2        2022-06-29T00:00:00.000
3        2021-09-11T00:00:00.000
4        2021-12-14T00:00:00.000
                  ...           
49995    2022-04-06T00:00:00.000
49996    2022-04-06T00:00:00.000
49997    2022-04-08T00:00:00.000
49998    2022-04-08T00:00:00.000
49999    2022-04-06T00:00:00.000
Name: crash_date, Length: 50000, dtype: object

In [12]:
# add a date string (foreign key)

df_s3['date_id'] = pd.to_datetime(df_s3['crash_date']).dt.strftime('%Y%m%d')
df_s3['date_id']

0        20210911
1        20220326
2        20220629
3        20210911
4        20211214
           ...   
49995    20220406
49996    20220406
49997    20220408
49998    20220408
49999    20220406
Name: date_id, Length: 50000, dtype: object

In [31]:
# create a df to upload to facts table:
facts_cols = [
    'collision_id',
    'date_id',
    'crash_time',
    'zip_code',
    'on_street_name',
    'off_street_name',
    'cross_street_name',
    'number_of_persons_injured',
    'number_of_persons_killed',
    'number_of_cyclist_injured',
    'number_of_cyclist_killed',
    'number_of_motorist_injured',
    'number_of_motorist_killed',
    'contributing_factor_vehicle_1',
    'contributing_factor_vehicle_2',
    'contributing_factor_vehicle_3',
    'contributing_factor_vehicle_4',
    'contributing_factor_vehicle_5',
	'vehicle_type_code_3'  ,
  'vehicle_type_code_4' ,
  'vehicle_type_code_5' ,
  'vehicle_type_code1' ,
  'vehicle_type_code2'
]
# df_s3.columns
df_facts = df_s3[facts_cols]

In [32]:
df_facts

,collision_id,date_id,crash_time,zip_code,on_street_name,off_street_name,cross_street_name,number_of_persons_injured,number_of_persons_killed,number_of_cyclist_injured,...,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,vehicle_type_code1,vehicle_type_code2
0,4455765,20210911,2024-05-21 02:39:00,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2,0,0,...,Aggressive Driving/Road Rage,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,Sedan,Sedan
1,4513547,20220326,2024-05-21 11:45:00,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1,0,0,...,Pavement Slippery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sedan,NaN
2,4541903,20220629,2024-05-21 06:55:00,NaN,THROGS NECK BRIDGE,NaN,NaN,0,0,0,...,Following Too Closely,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,Sedan,Pick-up Truck
3,4456314,20210911,2024-05-21 09:35:00,11208.0,NaN,NaN,1211 LORING AVENUE,0,0,0,...,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sedan,NaN
4,4486609,20211214,2024-05-21 08:13:00,11233.0,SARATOGA AVENUE,DECATUR STREET,NaN,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,4517165,20220406,2024-05-21 01:00:00,10065.0,2 AVENUE,EAST 67 STREET,NaN,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49996,4516760,20220406,2024-05-21 10:00:00,NaN,BROOKLYN QUEENS EXPRESSWAY,NaN,NaN,0,0,0,...,Unsafe Lane Changing,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,Station Wagon/Sport Utility Vehicle,Box Truck
49997,4517325,20220408,2024-05-21 18:15:00,10304.0,RICHMOND ROAD,ATLANTIC AVENUE,NaN,0,0,0,...,Following Too Closely,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle
49998,4517670,20220408,2024-05-21 15:17:00,NaN,LONG ISLAND EXPRESSWAY,NaN,NaN,0,0,0,...,Driver Inattention/Distraction,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle


In [ ]:
for i in df_s3.columns:
  if 'v' in i:
    print(i)

contributing_factor_vehicle_1
contributing_factor_vehicle_2
vehicle_type_code1
vehicle_type_code2
contributing_factor_vehicle_3
vehicle_type_code_3
contributing_factor_vehicle_4
vehicle_type_code_4
contributing_factor_vehicle_5
vehicle_type_code_5


Just noticed no victim column is in this dataset, even though it was included in the data dictionary. Yikes!

In [ ]:
# split the dataset into different columns
df_s3.columns

Index(['crash_date', 'crash_time', 'on_street_name', 'off_street_name',
       'number_of_persons_injured', 'number_of_persons_killed',
       'number_of_pedestrians_injured', 'number_of_pedestrians_killed',
       'number_of_cyclist_injured', 'number_of_cyclist_killed',
       'number_of_motorist_injured', 'number_of_motorist_killed',
       'contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'collision_id', 'vehicle_type_code1', 'vehicle_type_code2', 'borough',
       'zip_code', 'latitude', 'longitude', 'location', 'cross_street_name',
       'contributing_factor_vehicle_3', 'vehicle_type_code_3',
       'contributing_factor_vehicle_4', 'vehicle_type_code_4',
       'contributing_factor_vehicle_5', 'vehicle_type_code_5'],
      dtype='object')

In [ ]:
vehicle_types = ['contributing_factor_vehicle_1', 'contributing_factor_vehicle_2',
       'collision_id', 'vehicle_type_code1', 'vehicle_type_code2', 'borough',
       'zip_code', 'latitude', 'longitude', 'location', 'cross_street_name',
       'contributing_factor_vehicle_3', 'vehicle_type_code_3',
       'contributing_factor_vehicle_4', 'vehicle_type_code_4',
       'contributing_factor_vehicle_5', 'vehicle_type_code_5']
df_s3[vehicle_types]

,contributing_factor_vehicle_1,contributing_factor_vehicle_2,collision_id,vehicle_type_code1,vehicle_type_code2,borough,zip_code,latitude,longitude,location,cross_street_name,contributing_factor_vehicle_3,vehicle_type_code_3,contributing_factor_vehicle_4,vehicle_type_code_4,contributing_factor_vehicle_5,vehicle_type_code_5
0,Aggressive Driving/Road Rage,Unspecified,4455765,Sedan,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Pavement Slippery,NaN,4513547,Sedan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Following Too Closely,Unspecified,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Unspecified,NaN,4456314,Sedan,NaN,BROOKLYN,11208.0,40.667202,-73.866500,"{'latitude': '40.667202', 'longitude': '-73.86...",1211 LORING AVENUE,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,4486609,NaN,NaN,BROOKLYN,11233.0,40.683304,-73.917274,"{'latitude': '40.683304', 'longitude': '-73.91...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,NaN,NaN,4517165,NaN,NaN,MANHATTAN,10065.0,40.765614,-73.960700,"{'latitude': '40.765614', 'longitude': '-73.96...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
49996,Unsafe Lane Changing,Unspecified,4516760,Station Wagon/Sport Utility Vehicle,Box Truck,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49997,Following Too Closely,Unspecified,4517325,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,STATEN ISLAND,10304.0,40.594930,-74.097990,"{'latitude': '40.59493', 'longitude': '-74.097...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
49998,Driver Inattention/Distraction,Unspecified,4517670,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,NaN,NaN,40.755480,-73.741730,"{'latitude': '40.75548', 'longitude': '-73.741...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(df_s3['vehicle_type_code1'].nunique())
print(df_s3['vehicle_type_code2'].nunique())

209
268


In [ ]:
unique_in_code1 = []
unique_in_code2 = []

code1_lowercase = [x.lower() for x in df_s3['vehicle_type_code1'].unique()]
code2_lowercase = [x.lower() for x in df_s3['vehicle_type_code2'].unique()]

for i in code1_lowercase:
  if i not in code2_lowercase:
    unique_in_code1.append(i)

for i in code1_lowercase:
  if i not in code2_lowercase:
    unique_in_code2.append(i)


AttributeError: 'float' object has no attribute 'lower'

In [ ]:
import numpy as np

# Function to check if a value is NaN
is_nan = np.vectorize(lambda x: isinstance(x, float) and np.isnan(x))

# Convert to lowercase safely
vehicle_types_lower = [x.lower() if isinstance(x, str) else x for x in df_s3['vehicle_type_code1'] if not is_nan(x)]

In [ ]:
vehicle_types_lower.nunique()

AttributeError: 'list' object has no attribute 'nunique'

In [ ]:
df_s3['vehicle_type_code1'].unique()

array(['Sedan', nan, 'Dump', 'Station Wagon/Sport Utility Vehicle',
       'Tanker', 'Bus', 'Taxi', 'Van', 'Motorscooter', 'Bike', 'E-Bike',
       'Ambulance', 'Box Truck', 'Garbage or Refuse', 'Motorcycle',
       'E-Scooter', 'UTILITY', 'Pick-up Truck', 'Convertible',
       'Tractor Truck Diesel', 'Flat Bed', 'Tow Truck / Wrecker', '�MBU',
       'PK', 'Moped', 'AMBULANCE', 'Trailer', 'SCHOOL BUS', 'Lift Boom',
       'scooter', 'CATER', 'PKUP', 'Carry All', 'FDNY Ambul', '3-Door',
       'Beverage Truck', 'SEMI TRAIL', 'TOW TRUCK', 'Motorbike',
       'UNKNOWN', 'trailer', 'Lunch Wagon', 'dump', 'Flat Rack', 'UNK',
       'MOPED', 'USPCS', 'Tractor Truck Gasoline', 'GARBAGE TR',
       'Minicycle', 'FIRE TRUCK', 'Armored Truck', 'USPS truck', 'COM',
       'Chassis Cab', 'SW/VAN', 'Bulk Agriculture', 'Concrete Mixer',
       'Open Body', 'Excavator', 'Fork lift', '4 dr sedan', 'FORD VAN',
       'STREET SWE', 'unknown', 'fire truck', 'Glass Rack', 'COMMERCIAL',
       'PSD', 'PICK

In [ ]:
print("unique in code1:\n", unique_in_code1)
print("unique in code2:\n", unique_in_code2)

unique in code1:
 [nan, 'UTILITY', '�MBU', 'SCHOOL BUS', 'CATER', 'PKUP', 'FDNY Ambul', 'SEMI TRAIL', 'dump', 'USPCS', 'USPS truck', 'SW/VAN', 'Excavator', 'FORD VAN', 'Glass Rack', 'PSD', 'PICK UP', 'MTA', 'e bike uni', 'USPS VEHIC', 'E-bike', 'Truck', 'WORK VAN', 'MOTOR SCOO', 'CARRIER', 'DEPARTMENT', 'AMBULACE', '2 dr sedan', 'FDNY Firet', 'AMBULANE', 'MTR H', 'Self', 'GOLF CART', 'Mailtruck', 'Delivery', 'Pick up tr', 'EMS bus', 'freight', 'Postal ser', 'USPS Truck', 'CON ED TRU', 'SANMEN COU', 'PUMP', 'commercial', 'PICK UP TR', 'SHUTTLE BU', 'NYS Ambula', '12 passage', 'FORD', 'E scooter', 'dump truck', 'AMBULETTE', 'Garbage tr', 'GOLF CAR', 'UHAL', 'Livestock Rack', '.', 'Dodge', '50cc Scoot', 'FIRE APPAR', 'Dump truck', 'Mini Van', 'ROAD SWEEP', 'SELF INSUR', 'STREET CLE', 'Lift', 'Utility', 'EMS', 'kick scoot', 'LADDER CO', 'SEMI', 'Cargo van', 'Ambu', 'PUMPER', 'pas', 'TRUCK COMM', 'Cement tru', 'Ford Van', 'Skywatch', 'Carriage', 'Sprinter V', 'SPRINTER V', 'TL', 'MACK TRUCK

In [ ]:
# Generate date range from Jan 1, 2012 to May 1, 2024
date_range = pd.date_range(start='2012-01-01', end='2024-05-01', freq='D')

# Create DataFrame
df_datedim = pd.DataFrame({'date_id': date_range.strftime('%Y%m%d'),
                   'date': date_range,
                   'year': date_range.year,
                   'month': date_range.month,
                   'day': date_range.day})

# Display the DataFrame
df_datedim


,date_id,date,year,month,day
0,20120101,2012-01-01,2012,1,1
1,20120102,2012-01-02,2012,1,2
2,20120103,2012-01-03,2012,1,3
3,20120104,2012-01-04,2012,1,4
4,20120105,2012-01-05,2012,1,5
...,...,...,...,...,...
4500,20240427,2024-04-27,2024,4,27
4501,20240428,2024-04-28,2024,4,28
4502,20240429,2024-04-29,2024,4,29
4503,20240430,2024-04-30,2024,4,30


In [16]:
from google.oauth2 import service_account
#gclient = bigquery.Client()
credentials = service_account.Credentials.from_service_account_file('mydatawarehouseproject-45800c431e29.json')

# Initialize BigQuery client
gclient = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id
)

# List datasets in the project
datasets = list(gclient.list_datasets())

if datasets:
    print("Datasets in project:")
    for dataset in datasets:
        print(f"- {dataset.dataset_id}")

Datasets in project:
- dw2024_NYC_collision_data
- test_dataset_1


In [17]:
dataset_id = 'dw2024_NYC_collision_data'  # Replace with your dataset ID

# List tables in the dataset
tables = list(gclient.list_tables(dataset_id))

if tables:
    print("Tables in dataset:")
    for table in tables:
        print(f"- {table.table_id}")

Tables in dataset:
- crash_facts
- date_dimension
- location_dimension
- vehicle_dimension
- victim_dimension


In [ ]:
# Specify the dataset ID and table ID
dataset_id = 'dw2024_NYC_collision_data'  # Replace with your dataset ID
table_id = 'date_dimension'  # Replace with your table ID


# Specify the destination table
table_ref = gclient.dataset(dataset_id).table(table_id)

# Configure the job options
job_config = bigquery.LoadJobConfig()

# Load data into BigQuery table from DataFrame
job = gclient.load_table_from_dataframe(df_datedim, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

LoadJob<project=mydatawarehouseproject, location=US, id=286aef05-fda1-4c77-a643-2ec4b074a09d>

Send to facts table

In [45]:
# Specify the dataset ID and table ID
dataset_id = 'dw2024_NYC_collision_data'  # Replace with your dataset ID
table_id = 'crash_facts'  # Replace with your table ID


# Specify the destination table
table_ref = gclient.dataset(dataset_id).table(table_id)

# Configure the job options
job_config = bigquery.LoadJobConfig()

# Load data into BigQuery table from DataFrame
job = gclient.load_table_from_dataframe(df_facts, table_ref, job_config=job_config)

# Wait for the job to complete
job.result()

LoadJob<project=mydatawarehouseproject, location=US, id=d04602fe-7054-47aa-98c5-32a75f9037da>

In [21]:
df_facts['crash_time'] = df_facts['crash_time'].dt.time

<ipython-input-21-38e05e0a8cd1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facts['crash_time'] = df_facts['crash_time'].dt.time


In [43]:
df_facts['contributing_factor_vehicle_2']

0        Unspecified
1                NaN
2        Unspecified
3                NaN
4                NaN
            ...     
49995            NaN
49996    Unspecified
49997    Unspecified
49998    Unspecified
49999    Unspecified
Name: contributing_factor_vehicle_2, Length: 50000, dtype: object

In [34]:
for i in df_s3.columns:
  if 'coll' in i:
    print(i)

collision_id


In [41]:
#df_facts['collision_id'] = df_facts['collision_id'].astype(str)
df_facts['zip_code'] = df_facts['zip_code'].astype(str)

<ipython-input-41-d6ed404a16d5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facts['zip_code'] = df_facts['zip_code'].astype(str)


In [37]:
df_facts['collision_id']

0        4455765
1        4513547
2        4541903
3        4456314
4        4486609
          ...   
49995    4517165
49996    4516760
49997    4517325
49998    4517670
49999    4516845
Name: collision_id, Length: 50000, dtype: object

In [48]:
dataset_id = 'dw2024_NYC_collision_data'  # Replace with your dataset ID
table_id = 'crash_facts'  # Replace with your table ID
s3_bucket = 'datawarehouse2024'

for obj_num in range(2,43):
  # Specify the dataset ID and table ID

  response = s3.get_object(
    Bucket=s3_bucket,
    Key=f'dw_hw/nyc_crash_data/opendata_{obj_num}.json',
    )

  #print(response)

  df_s3 = pd.read_json(response['Body'])

  # add a date string (foreign key)

  df_s3['date_id'] = pd.to_datetime(df_s3['crash_date']).dt.strftime('%Y%m%d')

  df_facts = df_s3[facts_cols]

  # change from datetime to TIME
  df_facts['crash_time'] = df_facts['crash_time'].dt.time

  # change these types to string
  df_facts['collision_id'] = df_facts['collision_id'].astype(str)
  df_facts['zip_code'] = df_facts['zip_code'].astype(str)


  # Specify the destination table
  table_ref = gclient.dataset(dataset_id).table(table_id)

  # Configure the job options
  job_config = bigquery.LoadJobConfig()

  # Load data into BigQuery table from DataFrame
  job = gclient.load_table_from_dataframe(df_facts, table_ref, job_config=job_config)

  # Wait for the job to complete
  job.result()
  print(f'{i} object complete')

<ipython-input-48-0b7134d25d4e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facts['crash_time'] = df_facts['crash_time'].dt.time
<ipython-input-48-0b7134d25d4e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facts['collision_id'] = df_facts['collision_id'].astype(str)
<ipython-input-48-0b7134d25d4e>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

date_id object complete


<ipython-input-48-0b7134d25d4e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facts['crash_time'] = df_facts['crash_time'].dt.time
<ipython-input-48-0b7134d25d4e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facts['collision_id'] = df_facts['collision_id'].astype(str)
<ipython-input-48-0b7134d25d4e>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

date_id object complete


<ipython-input-48-0b7134d25d4e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facts['crash_time'] = df_facts['crash_time'].dt.time
<ipython-input-48-0b7134d25d4e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facts['collision_id'] = df_facts['collision_id'].astype(str)
<ipython-input-48-0b7134d25d4e>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

...

date_id object complete


<ipython-input-48-0b7134d25d4e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facts['crash_time'] = df_facts['crash_time'].dt.time
<ipython-input-48-0b7134d25d4e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facts['collision_id'] = df_facts['collision_id'].astype(str)
<ipython-input-48-0b7134d25d4e>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

date_id object complete


<ipython-input-48-0b7134d25d4e>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facts['crash_time'] = df_facts['crash_time'].dt.time
<ipython-input-48-0b7134d25d4e>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facts['collision_id'] = df_facts['collision_id'].astype(str)
<ipython-input-48-0b7134d25d4e>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https

date_id object complete
